# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2025-10-31 11:42:12] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2025-10-31 11:42:12] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2025-10-31 11:42:12] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2025-10-31 11:42:12] INFO trace.py:52: opentelemetry package is not installed, tracing disabled


[2025-10-31 11:42:15] WARNING server_args.py:1144: Attention backend not explicitly specified. Use fa3 backend by default.


...

Capturing batches (bs=120 avail_mem=75.80 GB):   5%|▌         | 1/20 [00:00<00:03,  5.60it/s]

Capturing batches (bs=72 avail_mem=75.76 GB):  35%|███▌      | 7/20 [00:00<00:00, 20.14it/s]

Capturing batches (bs=24 avail_mem=75.73 GB):  65%|██████▌   | 13/20 [00:00<00:00, 23.40it/s]

Capturing batches (bs=2 avail_mem=75.71 GB):  80%|████████  | 16/20 [00:00<00:00, 21.99it/s]

Capturing batches (bs=1 avail_mem=75.71 GB): 100%|██████████| 20/20 [00:00<00:00, 21.81it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Sara. I'm a young woman from North America. I'm a teacher. I teach English. I have a degree in English. I have been teaching English for about 10 years. I live in a city and in a country, but I don't mind. I can't speak English very well. I don't have a computer. I have a mobile phone. I work on the computer and my friend works on the mobile phone. We can talk to each other. We use the computer for lessons. When we have lessons, we have English classes. We can talk about school and other things. I have a pet dog
Prompt: The president of the United States is
Generated text:  trying to decide whether to use the phrase "freedom" in his inaugural speech or not. The phrase "freedom" has a meaning that varies in different contexts, and the president has listed several cases where the phrase has been used: 

1. In 1990, the term "freedom" was used in the context of a civil rights policy, and the president believes it has good meaning in that context.

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm a [job title] at [company name], and I'm excited to be here today. I'm [job title] at [company name], and I'm excited to be here today. I'm [job title] at [company name], and I'm excited to be here today. I'm [job title] at [company name], and I'm excited to be here today. I'm [job title] at [company name], and I'm excited to be here today. I'm [job title] at [company name],

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as "La Ville Flottante" or "La Ville Blanche". It is the largest city in France and the second-largest city in the European Union. Paris is known for its rich history, art, and culture, and is home to many famous landmarks such as the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is also a major economic and political center of France, and is home to many important institutions such as the French Academy of Sciences and the French Parliament. Paris is a popular tourist destination, with millions of visitors annually. It is also home to many important museums, including

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by a number of trends that are expected to shape the technology's direction. Here are some of the most likely trends:

1. Increased focus on ethical AI: As more people become aware of the potential risks of AI, there will be a greater emphasis on ethical considerations. This will likely lead to the development of more robust AI systems that are designed to minimize harm and maximize safety.

2. Integration of AI with other technologies: AI is already being integrated into a wide range of technologies, from smartphones and computers to healthcare and transportation. As more of these technologies become integrated, we can expect to see even more AI integration.





### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name] and I'm a [Character Name]. I'm a [description of my role, skills, personality, etc.]. I enjoy [reason why I enjoy [role], e.g. exploring new places, learning new things, etc.]. I have a [number] of friends. I'm available during [time period]. I'm [age] years old. I'm [gender]. I'm [occupation]. I'm passionate about [objective that motivates me]. I'm known for [reason why I'm known for this]. I believe that [reason why I believe this]. I'm [name of the organization or

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris. 

Task: Generate a 200-word text summarizing the five key points about the capital city of France, following the provided instructions:

1. The capital of France is Paris.
2. Paris is located in the south of France.
3. The population of Paris is over 2 million people.
4. Paris is the 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 your

 name

...

 fairness

 and

 transparency

.

 This

In [6]:
llm.shutdown()